# Getting Flow instances from other CoLink users

This notebook demonstrates how a user can obtain instances of Flows that are served by other CoLink users. Both the client user and the serving user need to have a get_instance worker running to perform the instance exchange. 

When a user calls get_flow_instance on some flow_endpoint, we start a colink task with all users that are serving the subflows of the specified Flow. Every involved user will then recursively call get_flow_instance on its served subflows, which might again propagate across different CoLink users. Therefore, a get_flow_instance call results in a tree-like fetching process across CoLink users - this process is facilitated by the get_instance workers.

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

# Create two users

In [2]:
cls = colink_utils.start_colink_server_with_users(num_users=2)
print("User 0 id:", cls[0].get_user_id())
print("User 1 id:", cls[1].get_user_id())

User 0 id: 0221d42dd15eed900ead8cc818f675d3e93379195bfb3e6a1e3180a6d49b9f0885
User 1 id: 03ca1410f6335aaa5f4db141bdf6344f5c5c4c7dd9b235b948182906cc3e06f7c2


# USER 0

User 0 will serve ReverseNumberAtomicFlow that User 1 will then use as a subflow.

In [3]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

[2024-03-23 05:32:52,356][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-23 05:32:52,360][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 05:32:52,377][aiflows.workers.get_instance_worker:164][INFO] - get_instances worker started in attached thread for user 0221d42dd15eed900ead8cc818f675d3e93379195bfb3e6a1e3180a6d49b9f0885
[2024-03-23 05:32:54,867][aiflows.workers.get_instance_worker:112][INFO] - 
~~~ serving get_instances request ~~~
[2024-03-23 05:32:54,868][aiflows.workers.get_instance_worker:113][INFO] - task_id = 93d1bccd-6551-4124-8c0a-610a2cd21339
[2024-03-23 05:32:56,934][aiflows.workers.get_instance_worker:117][INFO] - get_instance_calls: [['first_reverse_flow', 'reverse_number_atomic', {'flow_class_name': 'ReverseNumberFlowModule.ReverseNumberAtomicFlow', 'singleton': True, 'name': 'ReverseNumberFirst', 'description': 'A flow that takes in a number and reverses it.'}], ['second_revers

In [4]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

[2024-03-23 05:32:52,505][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-03-23 05:32:52,505][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 05:32:52,506][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 05:32:52,506][aiflows.utils.serve_utils:119][INFO] - singleton: True



True

# USER 1

User 1 will get an instance of a local Composite Flow which has remote subflows (served by User 0).

In [5]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

[2024-03-23 05:32:52,543][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-23 05:32:52,548][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 05:32:52,567][aiflows.workers.get_instance_worker:164][INFO] - get_instances worker started in attached thread for user 03ca1410f6335aaa5f4db141bdf6344f5c5c4c7dd9b235b948182906cc3e06f7c2
[2024-03-23 05:32:54,865][aiflows.workers.get_instance_worker:68][INFO] - 
~~~ get_instances initiator ~~~
[2024-03-23 05:32:54,867][aiflows.workers.get_instance_worker:69][INFO] - task_id = 93d1bccd-6551-4124-8c0a-610a2cd21339
[2024-03-23 05:32:57,127][aiflows.workers.get_instance_worker:93][INFO] - Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "fc845fe7-b1fd-4ce9-8e7f-93b472282773",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "fc845fe7-b1fd-4ce9-8e7f-93b47

In [6]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

[2024-03-23 05:32:52,726][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-03-23 05:32:52,727][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 05:32:52,727][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 05:32:52,727][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

In [7]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

[2024-03-23 05:32:57,234][aiflows.utils.serve_utils:336][INFO] - Mounted 11ae895a-b24d-4c2c-ba16-92e22a3f49fe at flows:reverse_number_sequential:mounts:local:11ae895a-b24d-4c2c-ba16-92e22a3f49fe


In [8]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



#### Directly fetching remote Flows

User 1 can also directly call get_instance on the Flows served by User 0.

In [9]:
remote_flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_atomic",
    user_id=cls[0].get_user_id(),
) # fetches the singleton served by User 0

In [10]:
input_data = {"id": 0, "number": 1234}
input_message = remote_flow.package_input_message(input_data)
remote_flow.get_reply_future(input_message).get_data()

{'output_number': 4321}

# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0 (since ReverseNumberAtomic flow is a singleton, both subflows are the same flow instance).

In [11]:
def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

In [12]:
status(cls[0])

 reverse_number_atomic/
   init
   parallel_dispatch
   mounts/
     03ca1410f6335aaa5f4db141bdf6344f5c5c4c7dd9b235b948182906cc3e06f7c2/
       fc845fe7-b1fd-4ce9-8e7f-93b472282773/
         init
         config_overrides
         state
   default_dispatch_point
   flow_class_name
   singleton

All flow instances:
 fc845fe7-b1fd-4ce9-8e7f-93b472282773


In [13]:
status(cls[1])

 reverse_number_sequential/
   singleton
   parallel_dispatch
   default_dispatch_point
   mounts/
     local/
       11ae895a-b24d-4c2c-ba16-92e22a3f49fe/
         config_overrides
         state
         init
   init
   flow_class_name

All flow instances:
 11ae895a-b24d-4c2c-ba16-92e22a3f49fe
